In [1]:
import pandas as pd

Get sample set for iModulon analytics

In [2]:
imoddb_ros_tale_samples = {'A1 F112 I1 R1',
 'A11 F83 I1 R1',
 'A13 F87 I1 R1',
 'A14 F27 I1 R1',
 'A16 F28 I1 R1',
 'A16 F32 I1 R1',
 'A17 F104 I1 R1',
 'A18 F24 I1 R1',
 'A18 F36 I1 R1',
 'A19 F91 I1 R1',
 'A21 F97 I0 R1',
 'A25 F111 I1 R1',
 'A29 F119 I1 R1',
 'A3 F76 I1 R1',
 'A4 F237 I1 R1',
 'A7 F103 I1 R1',
 'A9 F77 I1 R1'}

In [3]:
# raw_mut_df = pd.read_pickle('../all_aledb_raw_mut_df.pkl')
# raw_mut_df = raw_mut_df[(raw_mut_df['exp']=='ROS_eAMP')]
# raw_mut_df['afir'] = raw_mut_df.apply(lambda r: "A" + str(r.ale) + " F" + str(r.flask) + " I" + str(r.isolate) + " R" + str(r.tech_rep), axis=1)
# imoddb_ros_tale_mut_df = raw_mut_df[raw_mut_df['afir'].isin(imoddb_ros_tale_samples)]
# imoddb_oxyr_mut_df = imoddb_ros_tale_mut_df[imoddb_ros_tale_mut_df['Gene']=='oxyR']
# set(imoddb_oxyr_mut_df.afir)

Get variants for mutation 1D and 3D analytics

In [4]:
meta_muts = pd.read_pickle('./targ_mut_df.pkl')
display(len(meta_muts), meta_muts.head())

102

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest
466615,10mM_Lactose_R1,8 47 1 1,4159160,SNP,T→A,L224Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961}
303555,10mM_Lactose_R2,9 104 1 2,4158805,SNP,C→A,L106I (CTA→ATA),oxyR,private,True,{oxyR},{b3961},{b3961}
303558,10mM_Lactose_R2,9 46 1 2,4159070,SNP,T→A,L194Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961}
366871,10mM_Lactose_R3,10 6 1 3,4159121,SNP,C→A,A211D (GCC→GAC),oxyR,private,True,{oxyR},{b3961},{b3961}
353677,1mM_Gluc_10mM_Lactose_R1,4 49 1 1,4159087,SNP,T→G,L200V (TTG→GTG),oxyR,private,True,{oxyR},{b3961},{b3961}


In [5]:
meta_muts[meta_muts.exp=='ROS_eAMP']

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest
132903,ROS_eAMP,5 84 0 1,4158922,DEL,Δ6 bp,coding (433‐438/918 nt),oxyR,public,True,{oxyR},{b3961},{b3961}
131820,ROS_eAMP,11 20 1 1,4159147,SNP,C→G,R220G (CGC→GGC),oxyR,public,True,{oxyR},{b3961},{b3961}
131986,ROS_eAMP,13 87 1 1,4159126,SNP,G→A,A213T (GCG→ACG),oxyR,public,True,{oxyR},{b3961},{b3961}
132121,ROS_eAMP,17 104 0 1,4158785,SNP,C→T,P99L (CCC→CTC),oxyR,public,True,{oxyR},{b3961},{b3961}
132385,ROS_eAMP,21 97 0 1,4159079,SNP,G→T,G197V (GGT→GTT),oxyR,public,True,{oxyR},{b3961},{b3961}


In [6]:
# !!! Temporary; need to reprocess all ALE experiment data for this project with the now removed ERS project experiments which were being duplicated by the newer ROS_eAMP ALE project/experiment.

meta_muts = meta_muts[meta_muts.exp != 'GLU_16321-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_18361-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_4721-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']

In [7]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.mut import get_mut_size, get_mob_size, is_genetic_mut, is_coding_mut, predict_mutation_effect_on_feature, get_SNP_aa_pos, get_DEL_INS_MOB_aa_start_pos, get_DEL_AA_range


meta_muts["genetic"] = meta_muts["Details"].apply(is_genetic_mut)
meta_muts["coding"] = meta_muts["Details"].apply(is_coding_mut)
meta_muts["mutation size"] = meta_muts.apply(get_mut_size, axis=1)  # get_mut_size currently doesn't return MOB mut sizes. Isn't something currently necessary.

In [8]:
from aledbmutil.mut import get_SNP_aa_pos, get_DEL_AA_range

def _get_SNP_AA_range(mut_details_str):
    SNP_AA_range = ()
    AA_pos = get_SNP_aa_pos(mut_details_str)
    SNP_AA_range = (AA_pos, AA_pos)
    return SNP_AA_range

meta_muts["AA range"] = None
meta_muts["AA range"] = meta_muts.apply(lambda r:
                                        _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) \
                                        else (get_DEL_AA_range(r["Details"]) if ((r["Mutation Type"] == "DEL") & (r["coding"]==1)) \
                                        else r["AA range"]),
                                        axis=1)
meta_muts[meta_muts["Mutation Type"].isin(["SNP"])].head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range
466615,10mM_Lactose_R1,8 47 1 1,4159160,SNP,T→A,L224Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(224, 224)"
303555,10mM_Lactose_R2,9 104 1 2,4158805,SNP,C→A,L106I (CTA→ATA),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(106, 106)"
303558,10mM_Lactose_R2,9 46 1 2,4159070,SNP,T→A,L194Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(194, 194)"
366871,10mM_Lactose_R3,10 6 1 3,4159121,SNP,C→A,A211D (GCC→GAC),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(211, 211)"
353677,1mM_Gluc_10mM_Lactose_R1,4 49 1 1,4159087,SNP,T→G,L200V (TTG→GTG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(200, 200)"


In [9]:
# currently only tracking seq changes with SNPs for ALEdb mutations for later processing with AA property change trends
meta_muts['AA ref seq'] = meta_muts.apply(lambda m: m['Details'][0] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts['AA seq change'] = meta_muts.apply(lambda m: m['Details'].split(' (')[0][-1] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change
466615,10mM_Lactose_R1,8 47 1 1,4159160,SNP,T→A,L224Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(224, 224)",L,Q
303555,10mM_Lactose_R2,9 104 1 2,4158805,SNP,C→A,L106I (CTA→ATA),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(106, 106)",L,I
303558,10mM_Lactose_R2,9 46 1 2,4159070,SNP,T→A,L194Q (CTG→CAG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(194, 194)",L,Q
366871,10mM_Lactose_R3,10 6 1 3,4159121,SNP,C→A,A211D (GCC→GAC),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(211, 211)",A,D
353677,1mM_Gluc_10mM_Lactose_R1,4 49 1 1,4159087,SNP,T→G,L200V (TTG→GTG),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(200, 200)",L,V


In [10]:
exp_sel_pres_d = {
    'C321.∆A': "recoded stop codon",
    'C321.∆A.earlyfix': "recoded stop codon",
    'C321_A_earlyfix': "recoded stop codon",
    'Iron-ALE': "FeSO4",
    'JCL260__isobutanol_production_host-M9-MG1655gbk': 'isobutanol',
    'ROS_eAMP': "paraquat",  # starting strains were also pre-evolved on glucose
    'pgiPAE': 'Pyrobaculum aerophilum pgi',
    'pgiBME': 'Brucella melitensis pgi',
    'tpiHSA': 'Homo sapiens tpi',
    'tpiPAE': 'Pyrobaculum aerophilum tpi',
    'tpiVCH': 'Vibrio cholerae tpi',
 }


def _get_sel_pres(mut):
    sel_pres = ''
    if mut['ALEdb source'] == 'private':
        sel_pres = 'unpublished'
    else:
        if mut.exp in exp_sel_pres_d.keys():
            sel_pres = exp_sel_pres_d[mut.exp]        
    return sel_pres

meta_muts['selection pressure'] = meta_muts.apply(lambda r: _get_sel_pres(r), axis=1)
assert('' not in set(meta_muts['selection pressure']))

In [11]:
# TODO: fix warnings with append
import warnings
warnings.filterwarnings('ignore')


aa_var_df = pd.DataFrame()

from aledbmutil.mut import get_mut_size  #, get_DEL_INS_MOB_aa_start_pos


for mt, mut_type_mut_df in meta_muts.groupby("Mutation Type"):
    assert mt != "MOB", 'Currently not handilng MOB. The below INS logic might be able to handle this.'
    if mt == "SNP":
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = ""
            details_str = m["Details"]
            details_str = details_str[0:details_str.find(" (")]
            
            if details_str[-1] == '*':  # Could have them show the effected downstream positions like with ACS SynBio paper. 
                mut_eff = "truncation"
            elif details_str[0] == details_str[-1]:
                mut_eff = "synonymous substitution"
            elif details_str[0] != details_str[-1]:
                mut_eff = "nonsynonymous substitution"
            else:
                mut_eff = "unknown"
            
            aa_pos = int(details_str[1:-1])
            aa_var_df= aa_var_df.append({
                "AA pos": aa_pos,
                "mutation type": "SUB",
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': m['AA ref seq'],
                'AA seq change': m['AA seq change'],
            }, ignore_index=True)
    if (mt == 'DEL'):  # Unravel (unless truncation) since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        display(mut_type_mut_df)
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "deletion"
            mut_size = m['mutation size']
            if mut_size % 3 != 0:  # Don't want to record all positions involved in the truncation, just the first.
                aa_var_df= aa_var_df.append({
                    "AA pos": m['AA range'][0],
                    "mutation type": mt,
                    "mutation effect": "truncation",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)
            else:                
                for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                    aa_var_df= aa_var_df.append({
                        "AA pos": aa_pos,
                        "mutation type": mt,
                        "mutation effect": mut_eff,
                        "mutation source": m['selection pressure'],
                        'AA ref seq': '',
                        'AA seq change': '',
                    },ignore_index=True)
    if (mt == 'INS'):
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "insertion"  # currently assuming only insertions remain
            mut_size = m['mutation size']
            if mut_size % 3 != 0:
                mut_eff = "truncation"
            aa_var_df= aa_var_df.append({
                "AA pos": get_DEL_INS_MOB_aa_start_pos(m["Details"]),
                "mutation type": mt,
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': '',
                'AA seq change': '',
            },ignore_index=True)
    if (mt == 'SUB'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                aa_var_df= aa_var_df.append({
                    "AA pos": aa_pos,
                    "mutation type": mt,
                    "mutation effect": "nonsynonymous substitution",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)


display(len(aa_var_df), aa_var_df.head())

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change,selection pressure
29421,C321.∆A.earlyfix,1 1 1 1,4158776,DEL,Δ1 bp,coding (287/918 nt),oxyR,public,True,{oxyR},{b3961},{b3961},True,1,"(96, 96)",,,recoded stop codon
305292,C321_A_earlyfix,1 1 1 1,4158776,DEL,Δ1 bp,coding (287/918 nt),oxyR,private,True,{oxyR},{b3961},{b3961},True,1,"(96, 96)",,,unpublished
132903,ROS_eAMP,5 84 0 1,4158922,DEL,Δ6 bp,coding (433‐438/918 nt),oxyR,public,True,{oxyR},{b3961},{b3961},True,6,"(145, 146)",,,paraquat


98

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change
0,96,DEL,truncation,recoded stop codon,,
1,96,DEL,truncation,unpublished,,
2,145,DEL,deletion,paraquat,,
3,146,DEL,deletion,paraquat,,
4,224,SUB,nonsynonymous substitution,unpublished,L,Q


In [12]:
aa_var_cnt_df = pd.DataFrame()
gcols = ['AA pos', 'mutation type', 'mutation effect', 'mutation source', 'AA ref seq', 'AA seq change']
for g, gdf in aa_var_df.groupby(gcols):  # Essentially just finding duplicates
    d = dict(zip(gcols, g))
    d['mutation count'] = len(gdf)
    aa_var_cnt_df = aa_var_cnt_df.append(d, ignore_index=True)
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count
0,8,SUB,nonsynonymous substitution,recoded stop codon,Y,C,1
1,13,SUB,nonsynonymous substitution,recoded stop codon,A,V,1
2,29,SUB,nonsynonymous substitution,recoded stop codon,Q,R,1
3,29,SUB,nonsynonymous substitution,unpublished,Q,R,1
4,31,SUB,nonsynonymous substitution,recoded stop codon,T,M,1


In [13]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import const


aa_var_cnt_df['mutation effect color'] = aa_var_cnt_df['mutation effect'].apply(lambda e: const.MUTATION_TYPE_COLOR_PALETTE[e])
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color
0,8,SUB,nonsynonymous substitution,recoded stop codon,Y,C,1,#DD8452
1,13,SUB,nonsynonymous substitution,recoded stop codon,A,V,1,#DD8452
2,29,SUB,nonsynonymous substitution,recoded stop codon,Q,R,1,#DD8452
3,29,SUB,nonsynonymous substitution,unpublished,Q,R,1,#DD8452
4,31,SUB,nonsynonymous substitution,recoded stop codon,T,M,1,#DD8452


In [14]:
set(aa_var_cnt_df['mutation source'])

{'Brucella melitensis pgi',
 'FeSO4',
 'Homo sapiens tpi',
 'Pyrobaculum aerophilum pgi',
 'Pyrobaculum aerophilum tpi',
 'Vibrio cholerae tpi',
 'paraquat',
 'recoded stop codon',
 'unpublished'}

In [15]:
sel_pres_clr_d = {
    'FeSO4': '#729ECE',
    'Homo sapiens tpi': '#FF9E4A',
    'Pyrobaculum aerophilum pgi': '#67BF5C',
    'Brucella melitensis pgi': '#797979',
    'Pyrobaculum aerophilum tpi': '#ED665D',
    'Vibrio cholerae tpi': '#D5BB67',
    'isobutanol': '#ED97CA',
    'paraquat': '#6DCCDA',
    'recoded stop codon': '#A8786E',
    'unpublished': '#AD8BC9'
 }

aa_var_cnt_df['selection pressure color'] = aa_var_cnt_df['mutation source'].apply(lambda e: sel_pres_clr_d[e])
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color
0,8,SUB,nonsynonymous substitution,recoded stop codon,Y,C,1,#DD8452,#A8786E
1,13,SUB,nonsynonymous substitution,recoded stop codon,A,V,1,#DD8452,#A8786E
2,29,SUB,nonsynonymous substitution,recoded stop codon,Q,R,1,#DD8452,#A8786E
3,29,SUB,nonsynonymous substitution,unpublished,Q,R,1,#DD8452,#AD8BC9
4,31,SUB,nonsynonymous substitution,recoded stop codon,T,M,1,#DD8452,#A8786E
...,...,...,...,...,...,...,...,...,...
66,234,SUB,nonsynonymous substitution,unpublished,G,V,1,#DD8452,#AD8BC9
67,236,SUB,nonsynonymous substitution,Pyrobaculum aerophilum tpi,G,E,1,#DD8452,#ED665D
68,238,SUB,nonsynonymous substitution,Homo sapiens tpi,T,N,1,#DD8452,#FF9E4A
69,246,SUB,nonsynonymous substitution,unpublished,P,Q,1,#DD8452,#AD8BC9


In [16]:
def _get_variant_name(variant):
    var_name = ''
    if variant['mutation type'] == 'SUB':
        var_name = variant['mutation type'] + " " + variant['AA ref seq'] + str(variant['AA pos']) + variant['AA seq change']
    if variant['mutation type'] == 'DEL':
        var_name = variant['mutation type'] + " " + str(variant['AA pos'])
    return var_name


aa_var_cnt_df['name'] = aa_var_cnt_df.apply(lambda r: _get_variant_name(r), axis=1)
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color,name
0,8,SUB,nonsynonymous substitution,recoded stop codon,Y,C,1,#DD8452,#A8786E,SUB Y8C
1,13,SUB,nonsynonymous substitution,recoded stop codon,A,V,1,#DD8452,#A8786E,SUB A13V
2,29,SUB,nonsynonymous substitution,recoded stop codon,Q,R,1,#DD8452,#A8786E,SUB Q29R
3,29,SUB,nonsynonymous substitution,unpublished,Q,R,1,#DD8452,#AD8BC9,SUB Q29R
4,31,SUB,nonsynonymous substitution,recoded stop codon,T,M,1,#DD8452,#A8786E,SUB T31M
...,...,...,...,...,...,...,...,...,...,...
66,234,SUB,nonsynonymous substitution,unpublished,G,V,1,#DD8452,#AD8BC9,SUB G234V
67,236,SUB,nonsynonymous substitution,Pyrobaculum aerophilum tpi,G,E,1,#DD8452,#ED665D,SUB G236E
68,238,SUB,nonsynonymous substitution,Homo sapiens tpi,T,N,1,#DD8452,#FF9E4A,SUB T238N
69,246,SUB,nonsynonymous substitution,unpublished,P,Q,1,#DD8452,#AD8BC9,SUB P246Q


In [17]:
aa_var_cnt_df.to_csv('oxyR_ale_muts.csv', index=False)

In [18]:
mut_needle_plot_legend_df = pd.DataFrame()
mut_needle_plot_legend_df = aa_var_cnt_df.copy()
mut_needle_plot_legend_df = mut_needle_plot_legend_df[['mutation source', 'selection pressure color', ]]
mut_needle_plot_legend_df.rename(columns={'mutation source': 'name', 'selection pressure color': 'fill'}, inplace=True)
mut_needle_plot_legend_df['edge'] = 'white'
mut_needle_plot_legend_df.drop_duplicates(inplace=True)

df = aa_var_cnt_df.copy()
df = df[['mutation effect', 'mutation effect color']]
df.rename(columns={'mutation effect': 'name', 'mutation effect color': 'edge'}, inplace=True)
df.drop_duplicates(inplace=True)
df['fill'] = 'white'
mut_needle_plot_legend_df = mut_needle_plot_legend_df.append(df, ignore_index=True)
display(mut_needle_plot_legend_df)

mut_needle_plot_legend_df.to_csv('oxyR_ale_muts_legend.csv', index=False)

,name,fill,edge
0,recoded stop codon,#A8786E,white
1,unpublished,#AD8BC9,white
2,Brucella melitensis pgi,#797979,white
3,paraquat,#6DCCDA,white
4,Pyrobaculum aerophilum tpi,#ED665D,white
5,FeSO4,#729ECE,white
6,Pyrobaculum aerophilum pgi,#67BF5C,white
7,Homo sapiens tpi,#FF9E4A,white
8,Vibrio cholerae tpi,#D5BB67,white
9,nonsynonymous substitution,white,#DD8452


Creating functional feature annotation output

In [19]:
feats_df = pd.read_pickle('./annot_df.pkl')

def _rename_feat(f):
    new_name = f
    if f == 'The C-terminal substrate-binding domain of the LysR-type transcriptional regulator OxyR, a member of the type 2 periplasmic binding fold protein superfamily':
        new_name = 'C-terminal substrate-binding domain'
    if f == 'Winged helix DNA-binding domain superfamily':
        new_name = 'DNA binding domain'
    if f == 'LysR substrate binding domain':
        new_name = 'Substrate binding domain'
    return new_name

feats_df['feature'] = feats_df['feature'].apply(
    lambda f: _rename_feat(f)
)
# filtering out features that aren't useful
feats_to_remove = [
    'Initiator methionine',
    # 'Chain',
    'TRANSCRIPTIONAL DUAL REGULATOR HCAR-RELATED',
    'HYDROGEN PEROXIDE-INDUCIBLE GENES ACTIVATOR',
    'LysR, substrate-binding',
    '"Winged helix" DNA-binding domain',
    'Winged helix-like DNA-binding domain superfamily/Winged helix DNA-binding domain',
    'Winged helix-like DNA-binding domain superfamily',
    'Transcription regulator HTH, LysR',
    'Bacterial regulatory helix-turn-helix protein, lysR family',
    'LysR-type HTH domain profile',
    'C-terminal substrate-binding domain',
    'Periplasmic binding protein-like II',
    'HTHLYSR',
    'DNA binding',
    # 'HTH domain',
]
feats_df = feats_df[~(feats_df['feature'].isin(feats_to_remove))]
feats_df.reset_index(inplace=True, drop=True)
feats_df.feature.unique()

array(['Chain', 'Substrate binding domain', 'Disulfide bond region',
       'OxyR subunit interface', 'Modified residue', 'DNA binding domain',
       'HTH domain'], dtype=object)

In [20]:
layer = 0
layer_d = dict()
for f in feats_df.feature.unique():
    layer_d[f] = layer
    layer += 1
feats_df['layer'] = feats_df['feature'].apply(lambda f: layer_d[f])
feats_df.head()

,feature,start,end,range,layer
0,Chain,1,305,"(1, 305)",0
1,Substrate binding domain,89,293,"(89, 205)",1
2,Disulfide bond region,180,259,"(180, 80)",2
3,OxyR subunit interface,252,253,"(252, 2)",3
4,OxyR subunit interface,232,235,"(232, 4)",3


In [21]:
feat_color_d = dict()
muted=["#4878D0", "#6ACC64", "#D65F5F", "#956CB4", "#8C613C", "#DC7EC0", "#797979", "#D5BB67", "#82C6E2", "#EE854A"]  # from seaborn
feats = list(feats_df.feature.unique())
for i in range(0, len(feats_df.feature.unique())):
    feat_color_d[feats[i]] = muted[i]

feats_df['color'] = feats_df['feature'].apply(lambda f: feat_color_d[f])

feats_df.drop(columns=["range"], inplace=True)
feats_df.drop_duplicates(inplace=True)

display(feats_df, feat_color_d)
feats_df.to_csv('./oxyR_feats.csv')

,feature,start,end,layer,color
0,Chain,1,305,0,#4878D0
1,Substrate binding domain,89,293,1,#6ACC64
2,Disulfide bond region,180,259,2,#D65F5F
3,OxyR subunit interface,252,253,3,#956CB4
4,OxyR subunit interface,232,235,3,#956CB4
5,OxyR subunit interface,228,230,3,#956CB4
6,OxyR subunit interface,225,226,3,#956CB4
7,OxyR subunit interface,221,223,3,#956CB4
8,OxyR subunit interface,219,219,3,#956CB4
9,Disulfide bond region,199,208,2,#D65F5F


{'Chain': '#4878D0',
 'Substrate binding domain': '#6ACC64',
 'Disulfide bond region': '#D65F5F',
 'OxyR subunit interface': '#956CB4',
 'Modified residue': '#8C613C',
 'DNA binding domain': '#DC7EC0',
 'HTH domain': '#797979'}

In [22]:
# for NGL
ngl_feat_df = feats_df.copy()
ngl_feat_df["size"] = ngl_feat_df.apply(lambda r: r["end"] - r["start"] + 1, axis=1)
ngl_feat_df = ngl_feat_df[["feature", "start", "end", "color", "size"]]
ngl_feat_df = ngl_feat_df.sort_values("size", ascending=False)
ngl_feat_df = ngl_feat_df.reset_index(drop=True)  # json explicitly uses the indexes

with open('feats.json', 'w') as outfile:   # for NGL
    outfile.write(ngl_feat_df.to_json())
ngl_feat_df

,feature,start,end,color,size
0,Chain,1,305,#4878D0,305
1,Substrate binding domain,89,293,#6ACC64,205
2,DNA binding domain,1,86,#DC7EC0,86
3,Disulfide bond region,180,259,#D65F5F,80
4,HTH domain,1,58,#797979,58
5,Disulfide bond region,199,208,#D65F5F,10
6,OxyR subunit interface,120,126,#956CB4,7
7,OxyR subunit interface,122,126,#956CB4,5
8,OxyR subunit interface,232,235,#956CB4,4
9,OxyR subunit interface,221,223,#956CB4,3
